In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Mod4Milestone import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "123Abby456"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

# df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GSLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # add the unique identifier
    html.Center(html.B(html.H1('Hechmer Project Two Dashboard'))),
    # adding logo to the layout
    html.Hr(),
    html.Div([
    # add logo image and center it 
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         # adjust logo size for better display
                         style={'height':'30%', 'width':'30%'}))]),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # dropdown filter option
        dcc.Dropdown(
            id = 'filter_type',
            options = [
                {'label': 'Water Rescue', 'value': 'wRescue'},
                {'label': 'Mountain and Wilderness Rescue', 'value': 'mwRescue'},
                {'label': 'Disaster and Individual Tracking', 'value': 'diTrack'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'reset')
    ),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        # single row selection
        row_selectable = "single",
        ## limit number of rows per page
        page_size = 15,

    ),
    html.Br(),
    html.Hr(),
    ## format the graph and map accordingly
    html.Div(className = 'row', style={'display':'flex'}, children= [
        html.Div(id='map-id',
            className='col s12 m6',
            ),
        html.Div(id='graph-id',
            className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id','data'),
    Output('datatable-id', 'columns')],
    [Input('filter_type', 'value')]
)

def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
    # filter database for dogs that meet water rescue criteria
    if filter_type == 'wRescue':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever","Chesapeake Bay Retriever",
                                                                 "Newfoundland"]},
                             "sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gte":26},
                             "age_upon_outcome_in_weeks":{"$lte":156}})))
    
    # filter database for dogs that meet mountain/wilderness rescue
    elif filter_type == 'mwRescue':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute",
                                                                 "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                             "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":26},
                             "age_upon_outcome_in_weeks":{"$lte":156}})))
    
    # filter database for dogs that meet disaster/individual tracking criteria 
    elif filter_type == 'diTrack':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd", "Golden Retriever",
                                                                 "Bloodhound", "Rottweiler"]},
                             "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":20},
                             "age_upon_outcome_in_weeks":{"$lte":300}})))
    
    # reset filters
    elif filter_type == 'reset':
        df = pd.DataFrame.from_records(shelter.read({}))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records') 
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    # access the data from the chart
    dff = pd.DataFrame.from_dict(viewData)
    # create values needed for the pie chart display
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    # display the pie chart
    return [
        dcc.Graph(figure = px.pie(dff, names = names, values = values,
                                  title='Preferred Breeds Available'))]

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")]
) 

def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    # convert list to a row index to allow single row selection
    if index is None:
        row = 0
    else:
        row = index[0]
        
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width': '1000px', 'height': '500px'},
              center=[30.75,-97.48], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
                  # columns 13 and 14 define the grid coordinates
                  dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                           children=[
                               # column 4 define the breed 
                               # column 9 defines the name
                               dl.Tooltip(dff.iloc[row,4]),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(dff.iloc[row,9])
                               ])
                           ])
              ])
    ]
    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:8332/
